In [4]:
import torch
import numpy as np
from torch import nn
from kaggleguy import KaggleGuy2

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
device = "cpu"

In [7]:
model = KaggleGuy2(28, 47)
model.eval()
model.to(device)
state = torch.load("./models/KaggleGuy2-EMNIST_balanced8956")
model.load_state_dict(state["model"])

<All keys matched successfully>

In [11]:
emnist = [None] * 47

with open("./data/EMNIST/emnist_balanced_mapping.txt") as f:
    f = f.read()
    for line in f.split("\n"):
        line = line.split()
        emnist[int(line[0])] = int(line[1])    

def emnist_to_char(label):
    return chr(emnist[label])

In [14]:
chars = [chr(i) for i in emnist]
chars

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'd',
 'e',
 'f',
 'g',
 'h',
 'n',
 'q',
 'r',
 't']

In [15]:
import torch.onnx

In [17]:
model.eval()
x = torch.randn(256, 1, 28, 28, requires_grad=True).to(device)
torch_out = model(x)
torch.onnx.export(model, x, "kaggleguy2_emnist_balanced.onnx", export_params=True, do_constant_folding=True, input_names = ["input"], output_names = ["output"], dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}})
# onnx_program = torch.onnx.dynamo_export(model, x)

In [18]:
import onnx

In [19]:
onnx_model = onnx.load("kaggleguy2_emnist_balanced.onnx")
onnx.checker.check_model(onnx_model)

In [20]:
import onnxruntime

In [21]:
import numpy as np

In [22]:
x = np.random.rand(1, 1, 28, 28).astype("float32")

In [23]:
onnx_input = onnx_program.adapt_torch_inputs_to_onnx(x)
print(f"Input length: {len(onnx_input)}")
print(f"Sample input: {onnx_input}")

NameError: name 'onnx_program' is not defined

In [24]:
ort_session = onnxruntime.InferenceSession("kaggleguy2_emnist_balanced.onnx", providers=["CUDAExecutionProvider"])

In [25]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [26]:
onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}
onnxruntime_outputs = ort_session.run(None, onnxruntime_input)

NameError: name 'onnx_input' is not defined

In [27]:
ort_session.get_inputs()[0].name

'input'

In [28]:
onnxruntime_input = {"input": x}
onnxruntime_outputs = ort_session.run(None, onnxruntime_input)

In [59]:
torch_outputs = model(x)
torch_outputs = onnx_program.adapt_torch_outputs_to_onnx(torch_outputs)

In [29]:
assert len(torch_outputs) == len(onnxruntime_outputs)
for torch_output, onnxruntime_output in zip(torch_outputs, onnxruntime_outputs):
    torch.testing.assert_close(torch_output.to("cpu"), torch.tensor(onnxruntime_output), rtol=1e-3, atol=1e-5)

print("PyTorch and ONNX Runtime output matched!")
print(f"Output length: {len(onnxruntime_outputs)}")
print(f"Sample output: {onnxruntime_outputs}")

NameError: name 'torch_outputs' is not defined

In [30]:
import time

device = "cuda"
model.to(device)
x = torch.randn(256, 1, 28, 28, requires_grad=True).to(device)

start = time.time()
torch_out = model(x)
end = time.time()
print(f"Inference of Pytorch model used {end - start} seconds")

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
start = time.time()
ort_outs = ort_session.run(None, ort_inputs)
end = time.time()
print(f"Inference of ONNX model used {end - start} seconds")

Inference of Pytorch model used 0.42252373695373535 seconds
Inference of ONNX model used 0.013001203536987305 seconds


In [31]:
ort_outs

[array([[-129.12921 , -239.12625 , -116.36777 , ...,  -69.749664,
         -133.84784 , -110.82469 ],
        [-129.3731  , -338.3047  , -164.02852 , ...,  -81.786446,
         -155.29788 , -157.15498 ],
        [-106.09598 , -198.57664 ,  -98.421646, ...,  -55.10247 ,
         -110.198524,  -95.312675],
        ...,
        [-104.35375 , -218.34138 , -110.92261 , ...,  -67.35361 ,
         -131.36137 , -107.69084 ],
        [-117.647446, -249.63028 , -114.04317 , ...,  -61.937504,
         -144.511   , -110.879745],
        [-132.99767 , -267.40372 , -126.5995  , ...,  -74.62038 ,
         -149.04042 , -114.0175  ]], dtype=float32)]

In [32]:
torch.testing.assert_close(to_numpy(torch_out), ort_outs[0], rtol=1e-3, atol=1e-5)

AssertionError: Tensor-likes are not close!

Mismatched elements: 124 / 12032 (1.0%)
Greatest absolute difference: 0.055210113525390625 at index (144, 14) (up to 1e-05 allowed)
Greatest relative difference: 2.9656779766082764 at index (66, 16) (up to 0.001 allowed)

In [33]:
for torch_output, onnxruntime_output in zip(torch_out, ort_outs[0]):
    torch.testing.assert_close(torch_output.to("cpu"), torch.tensor(onnxruntime_output), rtol=1e-3, atol=1e-5)

AssertionError: Tensor-likes are not close!

Mismatched elements: 1 / 47 (2.1%)
Greatest absolute difference: 0.0017535686492919922 at index (16,) (up to 1e-05 allowed)
Greatest relative difference: 0.0016048387624323368 at index (16,) (up to 0.001 allowed)

In [34]:
onnxruntime.get_device()

'GPU'

In [ ]:
%env ONNX_MODE=cuda

env: ONNX_MODE=cuda


In [67]:
print(onnxruntime.get_available_providers())

['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [35]:
import EMNIST as Data

test = Data.test_dataloader(256)

In [36]:
print(ort_session.get_providers())

['CPUExecutionProvider']


In [37]:
correct = 0
total = 0

for images, labels in test:
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(images)}
    ort_outs = ort_session.run(None, ort_inputs)
    preds = np.argmax(ort_outs[0], axis=1)
    correct += (preds == labels.numpy()).sum()
    total += labels.size(0)

accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 89.76%


In [38]:
correct = 0
total = 0
model.eval()
for images, labels in test:
    images, labels = images.to(device), labels.to(device)
    
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(images)
    
    _, preds = torch.max(outputs, 1)
    correct += (preds == labels).sum().item()
    total += labels.size(0)

accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 89.75%
